# **Dependancies**

In [210]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.losses import LogCosh

# **Loading Data**

In [211]:
PATH_TO_DATA = "reg_interval1.csv"
data = pd.read_csv(PATH_TO_DATA)

In [212]:
data.head()

,difficulty3var,fee_to_rewardUSD,hashrate90std,mediantransactionvalue90trxUSD,mining_profitability30trx,price14wmaUSD,price30emaUSD,price3wmaUSD,price7smaUSD,price7wmaUSD,price90momUSD,top100cap90sma,transactionvalue30smaUSD,transactionvalue90emaUSD,priceUSD
0,-4.294967e+09,24.447,5.438008e+18,1.707,0.516,13714,14124,11780,13261,12798,5091.0,16.875,96961,85113,10747
1,-5.368709e+09,23.309,5.589062e+18,1.716,0.446,13332,13959,11405,12933,12373,5773.0,16.886,96445,85012,11561
2,-5.368709e+09,19.255,5.634595e+18,1.722,0.376,12982,13804,11425,12605,12030,5483.0,16.897,96146,85180,11560
3,-5.368709e+09,13.105,6.187523e+18,1.715,0.162,12322,13456,11694,11695,11707,5555.0,16.941,94913,84139,11217
4,-5.368709e+09,11.452,6.338603e+18,1.705,0.090,12038,13285,11136,11487,11484,5254.0,16.958,94620,83633,10800


In [213]:
data.shape

(866, 15)

In [220]:
X = data

In [236]:
X

,difficulty3var,fee_to_rewardUSD,hashrate90std,mediantransactionvalue90trxUSD,mining_profitability30trx,price14wmaUSD,price30emaUSD,price3wmaUSD,price7smaUSD,price7wmaUSD,price90momUSD,top100cap90sma,transactionvalue30smaUSD,transactionvalue90emaUSD,priceUSD
0,-4.294967e+09,24.447,5.438008e+18,1.707,0.516,13714,14124,11780,13261,12798,5091.0,16.875,96961,85113,10747
1,-5.368709e+09,23.309,5.589062e+18,1.716,0.446,13332,13959,11405,12933,12373,5773.0,16.886,96445,85012,11561
2,-5.368709e+09,19.255,5.634595e+18,1.722,0.376,12982,13804,11425,12605,12030,5483.0,16.897,96146,85180,11560
3,-5.368709e+09,13.105,6.187523e+18,1.715,0.162,12322,13456,11694,11695,11707,5555.0,16.941,94913,84139,11217
4,-5.368709e+09,11.452,6.338603e+18,1.705,0.090,12038,13285,11136,11487,11484,5254.0,16.958,94620,83633,10800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,0.000000e+00,7.662,2.259369e+19,0.239,-0.269,11180,10963,11421,11345,11400,2248.0,14.235,86920,70692,11417
862,1.374390e+11,6.213,2.227210e+19,0.236,-0.263,11230,10989,11392,11400,11406,2169.0,14.232,86438,70954,11370
863,1.191951e+22,3.487,2.261118e+19,0.233,-0.258,11270,11012,11367,11402,11393,2151.0,14.229,85452,71592,11348
864,8.458707e+22,4.637,2.262627e+19,0.230,-0.258,11313,11040,11395,11412,11401,2213.0,14.226,84709,72328,11433


In [215]:
#X = data.iloc[:,:-1]

In [216]:
#y=data.iloc[:,-1:]

In [218]:
y = data['priceUSD'].shift(-1,fill_value=1)

In [219]:
y

0      11561
1      11560
2      11217
3      10800
4      11094
       ...  
861    11370
862    11348
863    11433
864    11582
865        1
Name: priceUSD, Length: 866, dtype: int64

In [221]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, train_size=0.8, shuffle=True, random_state=7)

In [222]:
X_train.shape

(692, 15)

In [223]:
estimators=[]

In [224]:
estimators.append(['robust',RobustScaler()])

In [225]:
estimators.append(['mixmax',MinMaxScaler()])

In [226]:
scale=Pipeline(estimators,verbose=True)

In [227]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [228]:
X_test=scale.transform(X_test)

In [229]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [230]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [231]:
y_train=y_train.values

In [232]:
y_train=np.reshape(y_train, (y_train.shape[0],1,1))

In [233]:
y_test=y_test.values

In [234]:
y_test=np.reshape(y_test,(y_test.shape[0],1,1))

In [237]:
X_train.shape

(692, 1, 15)

In [238]:
X_train

array([[[1.68717216e-13, 2.86952897e-01, 6.31738238e-02, ...,
         8.02423045e-02, 2.00253448e-02, 3.13203301e-01]],

       [[1.44614757e-13, 1.96331805e-02, 7.74825760e-01, ...,
         1.99576087e-01, 1.49239656e-01, 7.54813703e-01]],

       [[7.21955219e-03, 3.96415173e-02, 8.73710263e-01, ...,
         1.35507491e-01, 3.49964635e-01, 5.74018505e-01]],

       ...,

       [[1.44614757e-13, 3.86411005e-02, 6.81518733e-01, ...,
         2.15605280e-01, 2.67240363e-01, 6.14528632e-01]],

       [[6.32689560e-14, 1.32972072e-02, 2.20769253e-01, ...,
         2.03453924e-01, 3.06421667e-01, 4.30982746e-01]],

       [[6.32689560e-14, 1.82992914e-02, 1.68618009e-01, ...,
         2.48036996e-01, 3.67632323e-01, 3.05076269e-01]]])

In [239]:
y_train.shape

(692, 1, 1)

In [241]:
y_train

array([[[ 6807]],

       [[ 9755]],

       [[ 8340]],

       [[ 9917]],

       [[10237]],

       [[ 9039]],

       [[ 9968]],

       [[10058]],

       [[ 8559]],

       [[ 9224]],

       [[11297]],

       [[ 7953]],

       [[10404]],

       [[10447]],

       [[ 6799]],

       [[10082]],

       [[ 6526]],

       [[ 6097]],

       [[ 6405]],

       [[ 7250]],

       [[11521]],

       [[ 7400]],

       [[11614]],

       [[10319]],

       [[ 3933]],

       [[ 8704]],

       [[ 9457]],

       [[ 8199]],

       [[ 7211]],

       [[ 6442]],

       [[10289]],

       [[ 3900]],

       [[ 7583]],

       [[ 8180]],

       [[ 8081]],

       [[10469]],

       [[11801]],

       [[ 9490]],

       [[ 3935]],

       [[ 7127]],

       [[10566]],

       [[ 8214]],

       [[11403]],

       [[10331]],

       [[ 6568]],

       [[11563]],

       [[ 9087]],

       [[ 7107]],

       [[ 6666]],

       [[ 8907]],

       [[ 7274]],

       [[ 7541]],

       [[ 81

In [240]:
X_test.shape

(174, 1, 15)

# **Model Architecture + Training**

In [242]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [243]:

adam = optimizers.Adam(learning_rate=lr_schedule(0), amsgrad=True)

Learning rate:  0.001


In [244]:
model = Sequential()
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
model.add(Dense(1))
model.compile(loss="log_cosh", optimizer=adam, metrics=['mae'])

E:\DS440proj\BTCpred\.venv\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [245]:
mcp_save = ModelCheckpoint('model/LSTM_reg_seven_new.keras', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')

In [246]:
#model.compile(optimizer='adam', loss='log_cosh', metrics=['accuracy'])
model.fit(X_train,y_train, epochs=5000, batch_size=32, validation_data=(X_test,y_test), callbacks=[mcp_save,earlyStopping])

Epoch 1/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 8205.1289 - mae: 8205.8232 - val_loss: 8465.8789 - val_mae: 8466.5723
Epoch 2/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 8281.9023 - mae: 8282.5957 - val_loss: 8381.6641 - val_mae: 8382.3564
Epoch 3/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 8105.4233 - mae: 8106.1162 - val_loss: 7991.0322 - val_mae: 7991.7261
Epoch 4/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 7682.5859 - mae: 7683.2798 - val_loss: 6845.1489 - val_mae: 6845.8428
Epoch 5/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 6260.3706 - mae: 6261.0635 - val_loss: 4152.8745 - val_mae: 4153.5679
Epoch 6/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 3079.3806 - mae: 3080.0737 - val_loss: 1497.0518 - val_mae: 1497.7417
Epoch 7/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 1304.7050 - mae: 1305.3981 - val_loss: 1310.8428 - val_mae: 1311.5359
Epoch 8/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 1144.7430 - mae: 1145.

# **Testing Model**

In [200]:
from tensorflow.keras.models import load_model
prediction_model = load_model('model/LSTM_reg_seven_new.keras',compile=False)

ValueError: A total of 2 objects could not be loaded. Example error message for object <LSTMCell name=lstm_cell, built=True>:

Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['kernel', 'recurrent_kernel', 'bias']

List of objects that could not be loaded:
[<LSTMCell name=lstm_cell, built=True>, <LSTMCell name=lstm_cell, built=True>]

In [247]:
y_pred = np.ravel(model.predict(X_test))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [248]:
y_test=np.ravel(y_test)

In [249]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.7650239534493695

In [250]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

238.52154190238866

In [251]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

906.149469033331

In [252]:
mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

6569.479838507135

In [253]:
pd.DataFrame(zip(['MAE','RMSE','MAPE','R^2'],[mae,rmse,mape,r2])).transpose()

,0,1,2,3
0,MAE,RMSE,MAPE,R^2
1,238.521542,906.149469,6569.479839,0.765024


In [254]:
pd.DataFrame([y_test,y_pred]).transpose()

,0,1
0,8706.0,8748.635742
1,9565.0,9546.569336
2,10164.0,10100.013672
3,7085.0,7082.275391
4,6454.0,6416.238281
...,...,...
169,7983.0,8274.499023
170,6167.0,6239.161133
171,11372.0,11603.534180
172,9233.0,9319.017578
